# P01: Replication Process

In [1]:
from dna import DNA
from replication import Replication
from dna_utils import DNAUtils

### Lectura de la cadena de ADN

In [2]:
dna_seq = ""
filename = "./data/Data1.txt"

with open(filename, 'r') as filename:
    for line in filename:
        dna_seq += line.strip()
        break

In [3]:
def write_results(filename, dna):
    with open(filename, 'w') as file:
        file.write(dna)

In [4]:
dna = DNA(dna_seq)
dna.show_nucleotides()

Number of nucleotides: 935


## Simulation process

In [5]:
replication = Replication()
dna_son1, dna_son2 = replication.execute(dna)

Number of helicases: 4
----------------------------------------------------------------------------------------------------
Helicases bind to the DNA strand and unwind the double helix at the positions: [0, 235, 446, 699]
Leading strand replication process...
Helicase unwinds the DNA double helix...
The unique primer of each the leading strand is placed at the 5' end of the DNA strand in positions:  [0, 235, 446, 699, 935]
----------------------------------------------------------------------------------------------------
The ADN polymerase enzyme is added to each primer and begins to synthesize the new DNA strand...
The leading strand replication process is complete.
----------------------------------------------------------------------------------------------------
Lagging strand replication process...
The lagging strand is synthesized in fragments called Okazaki fragments.
Helicases unwind the DNA double helix at the following positions:  [0, 235, 446, 699, 935]
--------------------

In [6]:
DNAUtils = DNAUtils()
test1 = DNAUtils.test_dna(dna.strand, dna_son2)
test2 = DNAUtils.test_dna(dna.comp, dna_son1)

if test1 and test2:
    print("The replication process has been successful.")
else:
    print("There's been a mutation in the replication process.")

write_results('./data/son_1.txt', dna_son1)
write_results('./data/son_2.txt', dna_son2)

The replication process has been successful.


## FAlta:
- animacion
- dejar presentable
- readme -> diagrama uml